In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


In [118]:
# Asegúrate de cargar el archivo 'caracteristicas_vinos.csv' en tu entorno de Google Colab
df = pd.read_csv('data_evaluacion.csv', header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [119]:
# Nuevas etiquetas para las columnas
nuevas_columnas = ['edad', 'clase_trabajo', 'fnlwgt', 'educacion', 'educacion_num', 'estado_civil','ocupacion', 'relacion', 
                   'raza', 'genero', 'ganancia-capital', 'perdida-capital', 'horas_semana', 'pais', 'salario']

# Asignar nuevas etiquetas
df.columns = nuevas_columnas
# Restablece el índice
df.reset_index(drop=True, inplace=True)


In [120]:
# Reemplazar los valores '?' por '0'
df.replace('?', 0, inplace=True)

In [121]:
df.head()

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   edad              48842 non-null  int64 
 1   clase_trabajo     48842 non-null  object
 2   fnlwgt            48842 non-null  int64 
 3   educacion         48842 non-null  object
 4   educacion_num     48842 non-null  int64 
 5   estado_civil      48842 non-null  object
 6   ocupacion         48842 non-null  object
 7   relacion          48842 non-null  object
 8   raza              48842 non-null  object
 9   genero            48842 non-null  object
 10  ganancia-capital  48842 non-null  int64 
 11  perdida-capital   48842 non-null  int64 
 12  horas_semana      48842 non-null  int64 
 13  pais              48842 non-null  object
 14  salario           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [123]:
# Elimina los espacios en blanco de los valores en la columna 'salary'
df['salario'] = df['salario'].str.strip()
salary_map = {'<=50K': 1, '>50K': 0}
df['salario'] = df['salario'].map(salary_map)


In [124]:
df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,1
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,1
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,0
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,0
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,0


In [125]:
# Elimina los espacios en blanco de los valores en la columna 'salary'
df['genero'] = df['genero'].str.strip()
# Mapea los valores
df['genero'] = df['genero'].map({'Male': 1, 'Female': 0})
print(df.head())

   edad     clase_trabajo  fnlwgt  educacion  educacion_num  \
0    39         State-gov   77516  Bachelors             13   
1    50  Self-emp-not-inc   83311  Bachelors             13   
2    38           Private  215646    HS-grad              9   
3    53           Private  234721       11th              7   
4    28           Private  338409  Bachelors             13   

         estado_civil          ocupacion       relacion   raza  genero  \
0       Never-married       Adm-clerical  Not-in-family  White       1   
1  Married-civ-spouse    Exec-managerial        Husband  White       1   
2            Divorced  Handlers-cleaners  Not-in-family  White       1   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black       1   
4  Married-civ-spouse     Prof-specialty           Wife  Black       0   

   ganancia-capital  perdida-capital  horas_semana           pais  salario  
0              2174                0            40  United-States        1  
1                 0   

In [126]:
df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,1
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,0,0,0,40,United-States,1
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,0,0,0,16,Jamaica,1
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,45,United-States,0
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,0,14084,0,50,United-States,0
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,5178,0,40,United-States,0


In [127]:
# Contar los registros de cada valor en la columna 'country'
country_counts = df['pais'].value_counts()
print(country_counts)


pais
United-States                 43832
Mexico                          951
0                               857
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru                   

In [128]:
# Elimina los espacios en blanco de los valores en la columna 'salary'
df['pais'] = df['pais'].str.strip()
data = [df]
for dataset in data:
    dataset.loc[dataset['pais'] != 'United-States', 'pais'] = 'Non-US'
    dataset.loc[dataset['pais'] == 'United-States', 'pais'] = 'US'

df.head(10)


,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,US,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,US,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,US,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,US,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Non-US,1
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,0,0,0,40,US,1
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,0,0,0,16,Non-US,1
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,45,US,0
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,0,14084,0,50,US,0
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,5178,0,40,US,0


In [129]:
df['pais'] = df['pais'].map({'US':1,'Non-US':0}).astype(int)
df.head(10)


,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,1,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,1,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,1,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,1,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,0,1
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,0,0,0,40,1,1
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,0,0,0,16,0,1
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,45,1,0
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,0,14084,0,50,1,0
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,5178,0,40,1,0


In [130]:
#Remplazamos estados civiles a Single y Couple
df['estado_civil'] = df['estado_civil'].str.strip()
df['estado_civil'] = df['estado_civil'].replace(['Divorced','Married-spouse-absent','Never-married','Separated','Widowed'],'Single')
df['estado_civil'] = df['estado_civil'].replace(['Married-AF-spouse','Married-civ-spouse'],'Couple')
df.head(10)


,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,Single,Adm-clerical,Not-in-family,White,1,2174,0,40,1,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Couple,Exec-managerial,Husband,White,1,0,0,13,1,1
2,38,Private,215646,HS-grad,9,Single,Handlers-cleaners,Not-in-family,White,1,0,0,40,1,1
3,53,Private,234721,11th,7,Couple,Handlers-cleaners,Husband,Black,1,0,0,40,1,1
4,28,Private,338409,Bachelors,13,Couple,Prof-specialty,Wife,Black,0,0,0,40,0,1
5,37,Private,284582,Masters,14,Couple,Exec-managerial,Wife,White,0,0,0,40,1,1
6,49,Private,160187,9th,5,Single,Other-service,Not-in-family,Black,0,0,0,16,0,1
7,52,Self-emp-not-inc,209642,HS-grad,9,Couple,Exec-managerial,Husband,White,1,0,0,45,1,0
8,31,Private,45781,Masters,14,Single,Prof-specialty,Not-in-family,White,0,14084,0,50,1,0
9,42,Private,159449,Bachelors,13,Couple,Exec-managerial,Husband,White,1,5178,0,40,1,0


In [131]:
df['estado_civil'] = df['estado_civil'].map({'Couple':0,'Single':1}) 
df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,1,Adm-clerical,Not-in-family,White,1,2174,0,40,1,1
1,50,Self-emp-not-inc,83311,Bachelors,13,0,Exec-managerial,Husband,White,1,0,0,13,1,1
2,38,Private,215646,HS-grad,9,1,Handlers-cleaners,Not-in-family,White,1,0,0,40,1,1
3,53,Private,234721,11th,7,0,Handlers-cleaners,Husband,Black,1,0,0,40,1,1
4,28,Private,338409,Bachelors,13,0,Prof-specialty,Wife,Black,0,0,0,40,0,1
5,37,Private,284582,Masters,14,0,Exec-managerial,Wife,White,0,0,0,40,1,1
6,49,Private,160187,9th,5,1,Other-service,Not-in-family,Black,0,0,0,16,0,1
7,52,Self-emp-not-inc,209642,HS-grad,9,0,Exec-managerial,Husband,White,1,0,0,45,1,0
8,31,Private,45781,Masters,14,1,Prof-specialty,Not-in-family,White,0,14084,0,50,1,0
9,42,Private,159449,Bachelors,13,0,Exec-managerial,Husband,White,1,5178,0,40,1,0


In [132]:
label_encoder = LabelEncoder()
df['relacion'] = label_encoder.fit_transform(df['relacion'])
df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,1,Adm-clerical,1,White,1,2174,0,40,1,1
1,50,Self-emp-not-inc,83311,Bachelors,13,0,Exec-managerial,0,White,1,0,0,13,1,1
2,38,Private,215646,HS-grad,9,1,Handlers-cleaners,1,White,1,0,0,40,1,1
3,53,Private,234721,11th,7,0,Handlers-cleaners,0,Black,1,0,0,40,1,1
4,28,Private,338409,Bachelors,13,0,Prof-specialty,5,Black,0,0,0,40,0,1
5,37,Private,284582,Masters,14,0,Exec-managerial,5,White,0,0,0,40,1,1
6,49,Private,160187,9th,5,1,Other-service,1,Black,0,0,0,16,0,1
7,52,Self-emp-not-inc,209642,HS-grad,9,0,Exec-managerial,0,White,1,0,0,45,1,0
8,31,Private,45781,Masters,14,1,Prof-specialty,1,White,0,14084,0,50,1,0
9,42,Private,159449,Bachelors,13,0,Exec-managerial,0,White,1,5178,0,40,1,0


In [133]:
# Obtener las categorías originales
categories = label_encoder.classes_

# Crear un DataFrame para mostrar el mapeo
mapping_df = pd.DataFrame({
    'Categoría': categories,
    'Código': range(len(categories))
})

print(mapping_df)

        Categoría  Código
0         Husband       0
1   Not-in-family       1
2  Other-relative       2
3       Own-child       3
4       Unmarried       4
5            Wife       5


In [134]:
label_encoder = LabelEncoder()
df['raza'] = label_encoder.fit_transform(df['raza'])
df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario
0,39,State-gov,77516,Bachelors,13,1,Adm-clerical,1,4,1,2174,0,40,1,1
1,50,Self-emp-not-inc,83311,Bachelors,13,0,Exec-managerial,0,4,1,0,0,13,1,1
2,38,Private,215646,HS-grad,9,1,Handlers-cleaners,1,4,1,0,0,40,1,1
3,53,Private,234721,11th,7,0,Handlers-cleaners,0,2,1,0,0,40,1,1
4,28,Private,338409,Bachelors,13,0,Prof-specialty,5,2,0,0,0,40,0,1
5,37,Private,284582,Masters,14,0,Exec-managerial,5,4,0,0,0,40,1,1
6,49,Private,160187,9th,5,1,Other-service,1,2,0,0,0,16,0,1
7,52,Self-emp-not-inc,209642,HS-grad,9,0,Exec-managerial,0,4,1,0,0,45,1,0
8,31,Private,45781,Masters,14,1,Prof-specialty,1,4,0,14084,0,50,1,0
9,42,Private,159449,Bachelors,13,0,Exec-managerial,0,4,1,5178,0,40,1,0


In [135]:
# Obtener las categorías originales
categories = label_encoder.classes_

# Crear un DataFrame para mostrar el mapeo
mapping_df = pd.DataFrame({
    'Categoría': categories,
    'Código': range(len(categories))
})

print(mapping_df)


            Categoría  Código
0  Amer-Indian-Eskimo       0
1  Asian-Pac-Islander       1
2               Black       2
3               Other       3
4               White       4


In [136]:
def definir_tipo_empleo(x):
    if x['clase_trabajo'] == 'Federal-gov' or x['clase_trabajo']== 'Local-gov' or x['clase_trabajo']=='State-gov': return 'govt'
    elif x['clase_trabajo'] == 'Private':return 'private'
    elif x['clase_trabajo'] == 'Self-emp-inc' or x['clase_trabajo'] == 'Self-emp-not-inc': return 'self_employed'
    else: return 'without_pay'
 
df['clase_trabajo'] = df['clase_trabajo'].str.strip()  
df['tipo_empleo']=df.apply(definir_tipo_empleo, axis=1)

df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario,tipo_empleo
0,39,State-gov,77516,Bachelors,13,1,Adm-clerical,1,4,1,2174,0,40,1,1,govt
1,50,Self-emp-not-inc,83311,Bachelors,13,0,Exec-managerial,0,4,1,0,0,13,1,1,self_employed
2,38,Private,215646,HS-grad,9,1,Handlers-cleaners,1,4,1,0,0,40,1,1,private
3,53,Private,234721,11th,7,0,Handlers-cleaners,0,2,1,0,0,40,1,1,private
4,28,Private,338409,Bachelors,13,0,Prof-specialty,5,2,0,0,0,40,0,1,private
5,37,Private,284582,Masters,14,0,Exec-managerial,5,4,0,0,0,40,1,1,private
6,49,Private,160187,9th,5,1,Other-service,1,2,0,0,0,16,0,1,private
7,52,Self-emp-not-inc,209642,HS-grad,9,0,Exec-managerial,0,4,1,0,0,45,1,0,self_employed
8,31,Private,45781,Masters,14,1,Prof-specialty,1,4,0,14084,0,50,1,0,private
9,42,Private,159449,Bachelors,13,0,Exec-managerial,0,4,1,5178,0,40,1,0,private


In [137]:
label_encoder = LabelEncoder()
df['tipo_empleo'] = label_encoder.fit_transform(df['tipo_empleo'])
df.head(10)

,edad,clase_trabajo,fnlwgt,educacion,educacion_num,estado_civil,ocupacion,relacion,raza,genero,ganancia-capital,perdida-capital,horas_semana,pais,salario,tipo_empleo
0,39,State-gov,77516,Bachelors,13,1,Adm-clerical,1,4,1,2174,0,40,1,1,0
1,50,Self-emp-not-inc,83311,Bachelors,13,0,Exec-managerial,0,4,1,0,0,13,1,1,2
2,38,Private,215646,HS-grad,9,1,Handlers-cleaners,1,4,1,0,0,40,1,1,1
3,53,Private,234721,11th,7,0,Handlers-cleaners,0,2,1,0,0,40,1,1,1
4,28,Private,338409,Bachelors,13,0,Prof-specialty,5,2,0,0,0,40,0,1,1
5,37,Private,284582,Masters,14,0,Exec-managerial,5,4,0,0,0,40,1,1,1
6,49,Private,160187,9th,5,1,Other-service,1,2,0,0,0,16,0,1,1
7,52,Self-emp-not-inc,209642,HS-grad,9,0,Exec-managerial,0,4,1,0,0,45,1,0,2
8,31,Private,45781,Masters,14,1,Prof-specialty,1,4,0,14084,0,50,1,0,1
9,42,Private,159449,Bachelors,13,0,Exec-managerial,0,4,1,5178,0,40,1,0,1


In [138]:
# Obtener las categorías originales
categories = label_encoder.classes_

# Crear un DataFrame para mostrar el mapeo
mapping_df = pd.DataFrame({
    'Categoría': categories,
    'Código': range(len(categories))
})

print(mapping_df)

       Categoría  Código
0           govt       0
1        private       1
2  self_employed       2
3    without_pay       3


KeyError: ('edad', 'tipo_empleo', 'fnlwgt', 'educacion_num', 'estado_civil', 'relacion', 'raza', 'genero', 'ganancia-capital', 'perdida-capital', 'horas_semana', 'pais', 'salario')